In [ ]:
import numpy as np
import time
from Helper import LearningCurvePlot, smooth
from Assignment2_batch_updates import *

def average_over_repetitions(policy, epsilon, smoothing_window, plot, decay_epsilon, n_repetitions, n_runs, learning_rate,
                             gamma, batch_size, experience_replay):

    reward_results = np.empty([n_repetitions,n_runs]) # Result array
    now = time.time()
    
    for rep in range(n_repetitions): # Loop over repetitions
        rewards = cartpole(n_runs, learning_rate, gamma, policy, epsilon, experience_replay, batch_size, decay_epsilon)
        reward_results[rep] = rewards
        
    print('Running one setting takes {} minutes'.format((time.time()-now)/60))    
    learning_curve = np.mean(reward_results,axis=0) # average over repetitions
    #learning_curve = smooth(learning_curve,smoothing_window) # additional smoothing
    return learning_curve  

def experiment():
    ####### Settings
    # Experiment    
    n_repetitions = 8
    smoothing_window = 1001
    n_runs = 100
    experience_replay = True
    gamma = 0.995
    optimal_average_reward_per_timestep = 300

    # Exploration
    policy = 'egreedy' # 'egreedy' or other choices
    epsilon_min = 0.01
    decay_rate = 0.95
    batch_size = 64
    decay_epsilon = False
    
    # Learning rate
    learning_rate = 0.01
    
    # Plotting parameters
    plot = True
    
    # Nice labels for plotting
    policy_labels = {'egreedy': '$\epsilon$-greedy policy',
                  'softmax': 'Softmax policy'}                  #Something here might need to be changed

    
    ####### Experiments
    
    good_average_reward = 350 # We set this as a benchmark of good average reward reached by the algorithm
    
    Plot = LearningCurvePlot(title = 'Cartpole experiment with $\epsilon$-greedy exploration')    
    policy = 'egreedy'
    epsilons = [0.01, 1 ,0.2]
    for epsilon in epsilons:
        if epsilon == 1:
            decay_epsilon = True
        learning_curve = average_over_repetitions(policy, epsilon, smoothing_window, plot, decay_epsilon, 
                                                  n_repetitions, n_runs, learning_rate, gamma, batch_size, experience_replay)
        Plot.add_curve(learning_curve,label=r'$\epsilon$-greedy, $\epsilon $ = {}'.format(epsilon))
        decay_epsilon = False
    #policy = 'softmax'
    #temps = [0.01,0.1,1.0]
    #for temp in temps:
     #   learning_curve = average_over_repetitions(backup, n_repetitions, n_timesteps, max_episode_length, learning_rate, 
      #                                        gamma, policy, epsilon, temp, smoothing_window, plot, n)
       # Plot.add_curve(learning_curve,label=r'softmax, $ \tau $ = {}'.format(temp))
    Plot.add_hline(optimal_average_reward_per_timestep, label = '')
    Plot.save('cartpole_test.png')

if __name__ == '__main__':
    experiment()


C:\Users\nikma\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:45: DeprecationWarning: invalid escape sequence \e


Run:1, Steps:10, Epsilon:0.01
Run:2, Steps:9, Epsilon:0.01
Run:3, Steps:9, Epsilon:0.01
Run:4, Steps:11, Epsilon:0.01
Run:5, Steps:10, Epsilon:0.01
Run:6, Steps:13, Epsilon:0.01
Run:7, Steps:9, Epsilon:0.01
Run:8, Steps:10, Epsilon:0.01
Run:9, Steps:10, Epsilon:0.01
Run:10, Steps:9, Epsilon:0.01
Run:11, Steps:9, Epsilon:0.01
Run:12, Steps:12, Epsilon:0.01
Run:13, Steps:8, Epsilon:0.01
Run:14, Steps:10, Epsilon:0.01
Run:15, Steps:10, Epsilon:0.01
Run:16, Steps:9, Epsilon:0.01
Run:17, Steps:9, Epsilon:0.01
Run:18, Steps:10, Epsilon:0.01
Run:19, Steps:9, Epsilon:0.01
Run:20, Steps:9, Epsilon:0.01
Run:21, Steps:12, Epsilon:0.01
Run:22, Steps:15, Epsilon:0.01
Run:23, Steps:11, Epsilon:0.01
Run:24, Steps:9, Epsilon:0.01
Run:25, Steps:24, Epsilon:0.01
Run:26, Steps:28, Epsilon:0.01
Run:27, Steps:15, Epsilon:0.01
Run:28, Steps:9, Epsilon:0.01
Run:29, Steps:8, Epsilon:0.01
Run:30, Steps:11, Epsilon:0.01
Run:31, Steps:10, Epsilon:0.01
Run:32, Steps:11, Epsilon:0.01
Run:33, Steps:11, Epsilon:0.01

Run:93, Steps:172, Epsilon:0.01
Run:94, Steps:155, Epsilon:0.01
Run:95, Steps:151, Epsilon:0.01
Run:96, Steps:150, Epsilon:0.01
Run:97, Steps:79, Epsilon:0.01
Run:98, Steps:94, Epsilon:0.01
Run:99, Steps:102, Epsilon:0.01
Run:100, Steps:142, Epsilon:0.01
Run:1, Steps:60, Epsilon:0.01
Run:2, Steps:78, Epsilon:0.01
Run:3, Steps:72, Epsilon:0.01
Run:4, Steps:13, Epsilon:0.01
Run:5, Steps:10, Epsilon:0.01
Run:6, Steps:10, Epsilon:0.01
Run:7, Steps:10, Epsilon:0.01
Run:8, Steps:9, Epsilon:0.01
Run:9, Steps:8, Epsilon:0.01
Run:10, Steps:8, Epsilon:0.01
Run:11, Steps:10, Epsilon:0.01
Run:12, Steps:10, Epsilon:0.01
Run:13, Steps:9, Epsilon:0.01
Run:14, Steps:10, Epsilon:0.01
Run:15, Steps:10, Epsilon:0.01
Run:16, Steps:25, Epsilon:0.01
Run:17, Steps:9, Epsilon:0.01
Run:18, Steps:10, Epsilon:0.01
Run:19, Steps:8, Epsilon:0.01
Run:20, Steps:8, Epsilon:0.01
Run:21, Steps:8, Epsilon:0.01
Run:22, Steps:9, Epsilon:0.01
Run:23, Steps:10, Epsilon:0.01
Run:24, Steps:10, Epsilon:0.01
Run:25, Steps:10, E